# `Part 1 - Aggregating data using SQLite and pandas`

This part consist in uploading all datasets into 1 database, to join, groupby and get our working dataset ready

We will use sqlite3 for its easy API, but also its friendly UI which enables to track changes and query the results very easily to ensure data quality 

In [2]:
import pandas as pd # pandas for data manipulation
import csv, sqlite3 # sqlite for sqlite local client interraction
import aws_creds # to store my AWS credentials without displaying in the notebook

import boto3 # for AWS
from io import StringIO # python3; python2: BytesIO 
 

## 1. Importing files

Files shown below were downloaded from<br>
*https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/*

In [3]:
#storing each file in pandas before injecting them in the database using SQL
#Locations file
loc_2019 = pd.read_csv('./source_data_locations/locations_2019.csv', sep=';')
loc_2020 = pd.read_csv('./source_data_locations/locations_2020.csv', sep=';')
loc_2021 = pd.read_csv('./source_data_locations/locations_2021.csv', sep=';')

#Users files
ppl_2019 = pd.read_csv('./source_data_people/people_involved_2019.csv', sep=';')
ppl_2020 = pd.read_csv('./source_data_people/people_involved_2020.csv', sep=';')
ppl_2021 = pd.read_csv('./source_data_people/people_involved_2021.csv', sep=';')


#Specificities
specs_2019 = pd.read_csv('./source_data_specs/specificities_2019.csv', sep=';')
specs_2020 = pd.read_csv('./source_data_specs/specificities_2020.csv', sep=';')
specs_2021 = pd.read_csv('./source_data_specs/specificities_2021.csv', sep=';')


#vehicules
vehi_2019 = pd.read_csv('./source_data_vehicules/vehicules_2019.csv', sep=';')
vehi_2020 = pd.read_csv('./source_data_vehicules/vehicules_2020.csv', sep=';')
vehi_2021 = pd.read_csv('./source_data_vehicules/vehicules_2021.csv', sep=';')

#### Establishing a connection with SQLite database

In [5]:
con = sqlite3.connect('db/capstone_sqlite.db') # creating a brand new database in sqlite to store our tables and schema 

In [6]:
cur = con.cursor() #our cursor to interact with the database (to query it, but also INSERT)

In [7]:
# importing our  tables into newly created sqlite database
#locations
loc_2019.to_sql('tb_locations_2019', con) 
loc_2020.to_sql('tb_locations_2020', con) 
loc_2021.to_sql('tb_locations_2021', con) 

#people
ppl_2019.to_sql('people_involved_2019', con)
ppl_2020.to_sql('people_involved_2020', con)
ppl_2021.to_sql('people_involved_2021', con)


#Specificities
specs_2019.to_sql('specificities_2019', con)
specs_2020.to_sql('specificities_2020', con)
specs_2021.to_sql('specificities_2021', con)

#vehicules
vehi_2019.to_sql('vehicules_2019', con)
vehi_2020.to_sql('vehicules_2020', con)
vehi_2021.to_sql('vehicules_2021', con)

97315

## 2. Appending data to the database

##### SQL script hidden here

In [8]:
# CREATE TABLE ppl AS
#   SELECT *
#   FROM people_involved_2019;  --all columns in people 2019,2020 and 2021 are the same
  
# CREATE TABLE specs AS
#   SELECT *
#   FROM specificities_2019; --all columns in specificities 2019,2020 and 2021 are the same

# CREATE TABLE vehi AS
#   SELECT *
#   FROM vehicules_2019 ; --all columns in vehicules 2019,2020 and 2021 are the same


# CREATE TABLE loc AS
#   SELECT *
#   FROM tb_locations_2019 ; --all columns in locations 2019,2020 and 2021 are the same


# --consolidating specs tables
# INSERT INTO specs SELECT * FROM specificities_2019;
# INSERT INTO specs SELECT * FROM specificities_2020;
# INSERT INTO specs SELECT * FROM specificities_2021;

# --consolidatings people_involved tables
# INSERT INTO ppl SELECT * FROM people_involved_2019;
# INSERT INTO ppl SELECT * FROM people_involved_2020;
# INSERT INTO ppl SELECT * FROM people_involved_2021;

# --consolidatings vehicule tables
# INSERT INTO vehi SELECT * FROM vehicules_2019;
# INSERT INTO vehi SELECT * FROM vehicules_2020;
# INSERT INTO vehi SELECT * FROM vehicules_2021;

# --consolidatings locations tables
# INSERT INTO loc SELECT * FROM tb_locations_2019;
# INSERT INTO loc SELECT * FROM tb_locations_2020;
# INSERT INTO loc SELECT * FROM tb_locations_2021;


# CREATE TABLE df_sql ('Num_Acc', 'id_vehicule', 'place', 'catu', 'grav', 'sexe', 'an_nais', 
#        'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp', 'senc', 
#        'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc', 'catr',
#        'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 
#        'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma', 'jour', 'mois', 
#        'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr',
#        'lat', 'long');
       
# INSERT INTO df_sql 
#     SELECT 
#         p.Num_Acc, p.id_vehicule, p.place, p.catu, p.grav, p.sexe, p.an_nais,p.trajet, p.secu1, p.secu2, p.secu3, p.locp, p.actp, p.etatp, 
#         v.senc, v.catv, v.obs, v.obsm, v.choc, v.manv, v.motor, v.occutc, 
#         l.catr,l.voie, l.v1, l.v2, l.circ, l.nbv, l.vosp, l.prof, l.pr, l.pr1, l.plan, l.lartpc, l.larrout, l.surf, l.infra, l.situ, l.vma, 
#         s.jour, s.mois,s.an, s.hrmn, s.lum, s.dep, s.com, s.agg, s.int, s.atm, s.col, s.adr,s.lat, s.long
#     FROM ppl p JOIN vehi v ON p.Num_Acc = v.Num_Acc AND p.id_vehicule = v.id_vehicule JOIN loc l ON p.Num_Acc = l.Num_Acc JOIN specs s ON p.Num_Acc = s.Num_Acc
#     GROUP BY p.Num_Acc, p.id_vehicule, p.place, p.catu, p.grav, p.sexe, p.an_nais,p.trajet, p.secu1, p.secu2, p.secu3, p.locp, p.actp, p.etatp;

##### Python code calling the script stored in a separate file:

In [9]:
#script to CREATE TABLES with each df headers. It also INSERT data, and multi JOIN them together
with open('db/sql_script_30112022.sql', 'r') as sql_file:
    sql_script = sql_file.read()

In [10]:
%%time
cur.executescript(sql_script) #executing the script

CPU times: total: 23.4 s
Wall time: 37 s


In [11]:
# fetch the table data after update, quick check with the first 2 records
print("Table data after updation :")
cur.execute("SELECT * FROM df_sql LIMIT 2") 
print(cur.fetchall())

Table data after updation :
[(201900000001, '138\xa0306\xa0524', 1, 1, 4, 2, 1993, 5, 1, 0, -1, -1, ' -1', -1, 2, 7, 0, 2, 5, 23, 1, None, 1, '3', 0.0, None, 3, 10, 0, 1, '6', '900', 2, None, None, 1, 2, 1, 70, 30, 11, 2019, '01:30', 4, '93', '93053', 1, 1, 1, 2, 'AUTOROUTE A3', '48,8962100', '2,4701200'), (201900000001, '138\xa0306\xa0524', 2, 2, 4, 2, 2002, 0, 1, 0, -1, -1, ' -1', -1, 2, 7, 0, 2, 5, 23, 1, None, 1, '3', 0.0, None, 3, 10, 0, 1, '6', '900', 2, None, None, 1, 2, 1, 70, 30, 11, 2019, '01:30', 4, '93', '93053', 1, 1, 1, 2, 'AUTOROUTE A3', '48,8962100', '2,4701200')]


In [12]:
# commit the changes and close the database 
con.commit()
cur.close()
con.close()

## 3. Back to pandas 

In this step we are going to pull the data from the database and use pandas dataframe to mainpulate it <br>
We will leverage the power of pandas to perform the EDA and features engineering. <br>
>From now on, `intermediate CSV files` will be saved at each milestone to store the data

In [13]:
#new df created from the master table (containing information from 4 datasets x 3 years) = 360k + records

#creating connection to the database
con = sqlite3.connect("db/capstone_sqlite.db") 

# from this database, target the table containing our aggregated 360k records and import that into a dataframe
df_from_sql = pd.read_sql_query('select * from df_sql', con) 

In [14]:
df_from_sql.shape # this ties back with our expectations

(367261, 53)

In [15]:
bucket = aws_creds.bucket # using the variable to load in AWS
csv_buffer = StringIO()
df_from_sql.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_from_sql.csv').put(Body=csv_buffer.getvalue()) # exporting the dataframe to a csv file in AWS S3 buckets

{'ResponseMetadata': {'RequestId': 'XNVD3YBW5QSR05DK',
  'HostId': 'BWCL9LbYg5GPxpykIicUePJnLY6rNYU5npOLra3r1jxYD4PuN4fyanwf08GvjG0Mc4nunFgS3OQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BWCL9LbYg5GPxpykIicUePJnLY6rNYU5npOLra3r1jxYD4PuN4fyanwf08GvjG0Mc4nunFgS3OQ=',
   'x-amz-request-id': 'XNVD3YBW5QSR05DK',
   'date': 'Tue, 06 Dec 2022 11:19:38 GMT',
   'etag': '"16aedcf29e04686c483186124bed6803"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"16aedcf29e04686c483186124bed6803"'}

In Part2, we will translate the data from french to english, and map the classes from numeric to english<br>
This will allow a meaningful EDA in Part3